In [1]:
import json
import numpy as np
from bokeh.plotting import figure
from bokeh import embed
from bokeh.io import show, output_notebook
from bokeh.palettes import Category20_20
colors = Category20_20
output_notebook()

Loading BokehJS ...

In [8]:
info = json.load(open('runs/40/info.json'))

In [9]:
c = 0
p = figure(plot_width=800, plot_height=600)

metrics = {}
for k in info['evaluations'][0].keys():
    if k.endswith('_auc'):
        metrics[k] = []

metrics['accuracy_test'] = []
for run in info['evaluations']:
    for m in metrics.keys():
        metrics[m].append(run[m])

for metric, values in metrics.items():
    xs = np.array([i for i in range(1, len(values)+1)])
    ys = np.array(values)
    legend = metric.replace('uncertainty_', '').replace('_auc', '')
    p.line(xs, ys, legend=legend, color=colors[c], line_width=3)
    c += 1

p.xaxis.axis_label = 'Train Data Size (%)'
p.yaxis.axis_label = 'AUC'
p.legend.location = 'bottom_right'

size = '14pt'
p.xaxis.major_label_text_font = size
p.xaxis.axis_label_text_font_size = size
p.yaxis.major_label_text_font = size
p.yaxis.axis_label_text_font_size = size

p.legend.label_text_font_size = size

show(p)

In [ ]:
c = 0
p = figure(title='CNN - '+ config['dataset_settings']['name'], plot_width=800, plot_height=600)
for k in sorted(info['evaluations'][0].keys()):
    if not k.startswith('uncertainty') or k.endswith('auc'): continue
    xys = np.array([info['evaluations'][i][k] for i in range(len(info['evaluations']))])
    xs = np.mean(xys, axis=0)[:, 0]
    ys = np.mean(xys, axis=0)[:, 1]
    stds = np.std(xys, axis=0)[:, 1]

    # Bollinger shading glyph:
    auc = np.array([info['evaluations'][i][k+'_auc'] for i in range(len(info['evaluations']))])
    auc = np.mean(auc)
    band_x = np.append(xs, xs[::-1])
    band_y = np.append(ys - stds, (ys + stds)[::-1])
    p.patch(band_x, band_y, color=colors[c], fill_alpha=0.2)
    p.line(xs, ys, legend='{0}: {1:.5f}'.format(k, auc), color=colors[c], line_width=3)
    c += 1

p.xaxis.axis_label = 'data proportion'
p.yaxis.axis_label = 'Accuracy'
p.legend.location = 'bottom_right'